In [1]:
# 0:clicks 1:carts 2:orders
type_weight = {0:1, 1:10, 2:3}
type_weight_multipliers = type_weight

top = 40
NUM_AIDS = 20000000

VER = 1
DIR = f'/home/anhphantq/vdb/covisit_ver{VER}'

In [2]:
import os 
try:
    os.mkdir(DIR)
    
    os.mkdir(DIR + '/splitted')
    os.mkdir(DIR + '/lb')
except OSError as error:
    print(error)    

[Errno 17] File exists: '/home/anhphantq/vdb/covisit_ver1'


In [3]:
import pandas as pd

# COMPUTE IN PARTS FOR MEMORY MANGEMENT

train_df = pd.read_parquet('/home/anhphantq/otto/splitted_data/train.parquet')
test_df = pd.read_parquet('/home/anhphantq/otto/splitted_data/test.parquet')
all_df = pd.concat((train_df, test_df))
MAX_TS = train_df.ts.max()
MIN_TS = train_df.ts.min()
MAX_TS, MIN_TS

(1661119199, 1659304800)

In [3]:
import pandas as pd
train_df = pd.read_parquet('/home/anhphantq/otto/data/train.parquet')

#remove 1 week from train df LB to retain distribution 
import numpy as np
min_ts = train_df.ts.values.min()
old_index = np.argwhere(train_df.ts.values - min_ts < 3600 * 24 * 7)
train_df = train_df.drop(old_index.flatten())
test_df = pd.read_parquet('/home/anhphantq/otto/data/test.parquet')
all_df = pd.concat((train_df, test_df))
MAX_TS = train_df.ts.max()
MIN_TS = train_df.ts.min()
MAX_TS, MIN_TS

(1661723999, 1659909600)

In [4]:
def create_covisit(df):    
    df = df.sort_values(['session','ts'],ascending=[True,False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 1 * 60 * 60) & (df.aid_x != df.aid_y) ]


    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y', 'type_y'])
    df['wgt'] = df.type_y.map(type_weight)
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    
    
    df = df.groupby(['aid_x','aid_y']).wgt.sum().reset_index().sort_values(['aid_x','wgt'],ascending=[True,False]).reset_index(drop = True)
    df['n'] = df.groupby('aid_x').aid_y.cumcount()
    #SAVE TOP 
    df = df.loc[df.n < top].drop('n', axis = 1)
    return df


In [5]:
from tqdm import tqdm
import numpy as np

step = all_df.shape[0] // 64
start = [i * step for i in range(64)]
end = [i + step for i in start]
end[-1] = all_df.shape[0] 

batches = [all_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]




In [6]:
%%time
#splitted

import multiprocessing
with multiprocessing.Pool(64) as pool:
  pd.concat(pool.map(create_covisit, batches, chunksize = 1)).to_parquet(DIR + f'/splitted/covisit')
  
  


CPU times: user 41.8 s, sys: 41.3 s, total: 1min 23s
Wall time: 1min 36s


In [6]:
%%time
#splitted

import multiprocessing
with multiprocessing.Pool(64) as pool:
  pd.concat(pool.map(create_covisit, batches, chunksize = 1)).to_parquet(DIR + f'/lb/covisit')
  
  


create top clicks


100%|██████████| 64/64 [01:28<00:00,  1.38s/it]


create top buy2buy


100%|██████████| 64/64 [00:15<00:00,  4.07it/s]


create top clicks


100%|██████████| 64/64 [00:15<00:00,  4.04it/s]


RECENT covisit

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from tqdm import tqdm
def create_recent_7_days(df):
    df['date'] = pd.to_datetime(df.ts * 1e9)#7 recent days
    df['dayofyear'] = df.date.dt.dayofyear

    dayofyear_max = df['dayofyear'].max()
    dayofyear_min = dayofyear_max - 6

    df = df.loc[(df.dayofyear >= dayofyear_min) & (df.dayofyear <= dayofyear_max)]
    result = []
    for i in range(dayofyear_min, dayofyear_max + 1):
        result.append(create_covisit(df.loc[df.dayofyear == i]))

    return result
        
   
def pqt_to_dict(df, i, f):
    df['covistation_list'] = list(zip(df.aid_y, df.wgt))
    
    df = df.groupby('aid_x').covistation_list.apply(list)
    disk_df = np.memmap(f'{f}/covisit{i}.array', dtype='object', mode='w+', shape=NUM_AIDS)
    disk_df[df.index.values] = df.values
    del df 
    return disk_df


    

In [7]:
from tqdm import tqdm
import numpy as np

step = train_df.shape[0] // 64
start = [i * step for i in range(64)]
end = [i + step for i in start]
end[-1] = train_df.shape[0] 

batches = [train_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]




In [12]:
%%time
#splitted

import multiprocessing

with multiprocessing.Pool(64) as pool:
  result = pool.map(create_recent_7_days, batches, chunksize = 1)
  for i in range(7):  
      pqt_to_dict(pd.concat([r[i] for r in result]), i, '/home/anhphantq/otto/splitted')





CPU times: user 33.1 s, sys: 30.7 s, total: 1min 3s
Wall time: 1min 9s


In [8]:
%%time
#lb

import multiprocessing

with multiprocessing.Pool(64) as pool:
  result = pool.map(create_recent_7_days, batches, chunksize = 1)
  for i in range(7):  
      pqt_to_dict(pd.concat([r[i] for r in result]), i, '/home/anhphantq/otto/lb')





CPU times: user 33.7 s, sys: 26.8 s, total: 1min
Wall time: 1min 9s
